In [7]:
from scrapy import project, signals
from scrapy.conf import settings
from scrapy.crawler import CrawlerProcess
from scrapy.xlib.pydispatch import dispatcher
from multiprocessing.queues import Queue
import multiprocessing


class CrawlerWorker(multiprocessing.Process):

    def __init__(self, spider, result_queue):
        multiprocessing.Process.__init__(self)
        self.result_queue = result_queue

        self.crawler = CrawlerProcess(settings)
        if not hasattr(project, 'crawler'):
            self.crawler.install()
        self.crawler.configure()

        self.items = []
        self.spider = spider
        dispatcher.connect(self._item_passed, signals.item_passed)

    def _item_passed(self, item):
        self.items.append(item)

    def run(self):
        self.crawler.crawl(self.spider)
        self.crawler.start()
        self.crawler.stop()
        self.result_queue.put(self.items)

def main():
    result_queue = Queue()
    crawler = CrawlerWorker(MySpider(myArgs), result_queue)
    crawler.start()
    for item in result_queue.get():
        yield item
    
if __name__ == '__main__':
    main()

ImportError: cannot import name 'project' from 'scrapy' (/opt/homebrew/Caskroom/miniconda/base/envs/py312/lib/python3.12/site-packages/scrapy/__init__.py)